# Подключение диска и установка keras.tunes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 9.3 MB/s eta 0:00:00


# Инициализация

In [17]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Dropout, LSTM, Bidirectional, SpatialDropout1D, Flatten, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from tensorflow import keras
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
# Максимальное количество слов
num_words = 10000
# Максимальная длина экшена
max_action_len = 50
#Число классов
num_classes = 15

# Загрузка тренировочных данных

In [5]:
train = pd.read_csv('drive/MyDrive/actionsNPCDataSet/trainActions.csv',header=None,
names=['class', 'text'])

In [6]:
#Получение экшенов
actions = train['text']

In [7]:
y_train = utils.to_categorical(train['class'] - 1, num_classes)

# Токенизация

In [8]:
tokenizer = Tokenizer(num_words=num_words)

In [9]:
#Обучение токенайзера
tokenizer.fit_on_texts(actions)

In [10]:
train_sequences = tokenizer.texts_to_sequences(actions)

In [11]:
x_train = pad_sequences(train_sequences, maxlen=max_action_len)

# Оптимизация гиперпараметров

In [15]:
def build_model(hp):
    activation_choice = hp.Choice('activation', values=['sigmoid', 'tanh', 'relu', 'elu', 'selu'])
    optimizer_choice = hp.Choice('optimizer', values=['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'])
    layers_conv_num = hp.Int('layers_num', 1, 3)
    neurons_num = hp.Int('neurons_num', min_value=12, max_value=256, step=32)
    filter_num = hp.Int('filter_num', min_value=32, max_value=512, step=32)
    kernel_size = hp.Int('kernel_size', min_value=1, max_value=5, step=1)
    embedding_num = hp.Int('embedding_num', min_value=4, max_value=128, step=16)
    embeddings_regularizer = keras.regularizers.l2(
        hp.Choice('embeddings_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))
    kernel_regularizer = keras.regularizers.l2(hp.Choice('kernel_regularizer', values=[1e-2, 1e-3, 1e-4], default=1e-3))

    model = Sequential()
    model.add(Embedding(input_dim=num_words, output_dim=embedding_num, input_length=max_action_len,
                        embeddings_regularizer=embeddings_regularizer))
    for i in range(layers_conv_num):
        model.add(Conv1D(filters=filter_num, kernel_size=kernel_size, activation=activation_choice, kernel_regularizer=kernel_regularizer))
        model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=neurons_num, activation=activation_choice, kernel_regularizer=kernel_regularizer))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=optimizer_choice, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [18]:
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=75,
    directory='cnn_actions_dir'
)

In [19]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
activation (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'tanh', 'relu', 'elu', 'selu'], 'ordered': False}
optimizer (Choice)
{'default': 'SGD', 'conditions': [], 'values': ['SGD', 'adam', 'adagrad', 'adadelta', 'rmsprop'], 'ordered': False}
layers_num (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
neurons_num (Int)
{'default': None, 'conditions': [], 'min_value': 12, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filter_num (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
embedding_num (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
embeddings_regularizer (Choice)
{'default': 0.001, 'conditions': [], 'values

In [20]:
tuner.search(x_train, y_train, batch_size=128, epochs=5, validation_split=0.2)

Trial 75 Complete [00h 00m 06s]
val_accuracy: 0.9688164591789246

Best val_accuracy So Far: 0.9858256578445435
Total elapsed time: 00h 09m 11s


In [21]:
tuner.results_summary(num_trials=74)

Results summary
Results in cnn_actions_dir/untitled_project
Showing 74 best trials
Objective(name="val_accuracy", direction="max")

Trial 67 summary
Hyperparameters:
activation: selu
optimizer: rmsprop
layers_num: 1
neurons_num: 172
filter_num: 64
kernel_size: 3
embedding_num: 116
embeddings_regularizer: 0.0001
kernel_regularizer: 0.0001
Score: 0.9858256578445435

Trial 44 summary
Hyperparameters:
activation: selu
optimizer: rmsprop
layers_num: 1
neurons_num: 236
filter_num: 32
kernel_size: 3
embedding_num: 100
embeddings_regularizer: 0.001
kernel_regularizer: 0.0001
Score: 0.9801559448242188

Trial 30 summary
Hyperparameters:
activation: selu
optimizer: rmsprop
layers_num: 1
neurons_num: 172
filter_num: 224
kernel_size: 3
embedding_num: 100
embeddings_regularizer: 0.001
kernel_regularizer: 0.001
Score: 0.9780297875404358

Trial 51 summary
Hyperparameters:
activation: selu
optimizer: rmsprop
layers_num: 1
neurons_num: 236
filter_num: 32
kernel_size: 5
embedding_num: 116
embeddings_regu

# Загрузка тестовых данных

In [22]:
test_actions = pd.read_csv('drive/MyDrive/actionsNPCDataSet/actionsTest.csv',header=None,
names=['class', 'text'])

In [23]:
#Получение экшенов
test_actions = train['text']

In [24]:
y_test = utils.to_categorical(train['class'] - 1, num_classes)

# Токенизация

In [25]:
test_sequences = tokenizer.texts_to_sequences(test_actions)

In [26]:
x_test = pad_sequences(test_sequences, maxlen=max_action_len)

# Проверка лучшей модели

In [27]:
model = tuner.get_best_models(num_models=1)

In [28]:
model[0].evaluate(x_test, y_test)

221/221 [==============================] - 1s 3ms/step - loss: 0.1470 - accuracy: 0.9972


[0.14698053896427155, 0.9971635341644287]

# Сохранение модели

In [29]:
model[0].save('drive/MyDrive/actionsNPCDataSet/actionsCNNModel.h5')